In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
import csv
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score

# 检测GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'使用 GPU: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')
    print('未检测到 GPU，使用 CPU')

# 加载专为中文优化的句向量模型
model = SentenceTransformer('shibing624/text2vec-base-chinese')
model = model.to(device)

print("CUDA 是否可用:", torch.cuda.is_available())
print("当前 PyTorch 版本:", torch.__version__)
print("CUDA 版本:", torch.version.cuda if torch.cuda.is_available() else "无")

使用 GPU: NVIDIA GeForce RTX 4050 Laptop GPU
CUDA 是否可用: True
当前 PyTorch 版本: 2.6.0+cu118
CUDA 版本: 11.8


## 读取训练集并做基础统计

In [3]:
import pandas as pd
train_df = pd.read_csv('csv/test.csv', sep='\t', header=None, names=['q1', 'q2', 'label'])
print('数据集样本数:', len(train_df))
print('正负样本分布:')
print(train_df['label'].value_counts())
train_df.head(10)

数据集样本数: 4401
正负样本分布:
label
1    2237
0    2164
Name: count, dtype: int64


,q1,q2,label
0,"跪求,英雄联盟好玩还是梦三国好玩?坐等呢","亲说下,英雄联盟好玩还是梦三国好玩?十分多谢",1
1,英雄联盟统治战场一天可以得多少经验,英雄联盟打统治战场会增加隐藏分吗,0
2,那里有免费的单机游戏下载,哪里有免费的单机游戏下载,1
3,平安夜的时候送女朋友什么礼物好呢？,今年的平安夜快要到了，送女朋友什么礼物好呢？,1
4,游戏王游戏在哪下载？,游戏王游戏在哪里下载,1
5,龙飞凤舞不一般的生肖是什么？,龙飞凤舞是什么生肖？,0
6,女生不喜欢肌肉男吗？,女生都喜欢肌肉男吗？,0
7,繁体字赛，宝贝怎么写,偶的宝贝繁体字怎么写啊,0
8,这手机怎么样，好不好用啊。,充电宝怎么样才算充满,0
9,魔兽世界燃铁矿哪里多,魔兽世界-魔铁矿石哪里多,0


## - 相似度分布可视化

## 高效批量推理：大幅加速相似度计算
将所有问题对批量编码，利用向量化操作一次性计算余弦相似度，大幅提升推理速度，适合大数据量场景。

In [9]:
## 避免显存溢出：分批计算余弦相似度
#一次性计算全部对角线会导致显存溢出。可以分批处理（如每1万对），每批计算后拼接结果，显著降低显存占用。

In [4]:
# 批量编码所有问题对
q1_list = train_df['q1'].tolist()
q2_list = train_df['q2'].tolist()

# 批量编码（可指定 batch_size，适当调大可加速）
q1_emb = model.encode(q1_list, convert_to_tensor=True, batch_size=128, show_progress_bar=True)
q2_emb = model.encode(q2_list, convert_to_tensor=True, batch_size=128, show_progress_bar=True)

# 分批计算对角线相似度
batch_size = 8000
sims = []
for start in range(0, len(q1_list), batch_size):
    end = min(start + batch_size, len(q1_list))
    emb1 = q1_emb[start:end]
    emb2 = q2_emb[start:end]
    sim = util.cos_sim(emb1, emb2).diagonal().cpu().numpy()
    sims.append(sim)
sims = np.concatenate(sims)

# 设定阈值，批量判断
threshold = 0.84
train_df['pred'] = (sims > threshold).astype(int)

# 计算指标
accuracy = accuracy_score(train_df['label'], train_df['pred'])
f1 = f1_score(train_df['label'], train_df['pred'])
recall = recall_score(train_df['label'], train_df['pred'])
print(f'训练集 Accuracy: {accuracy:.4f}')
print(f'训练集 F1-score: {f1:.4f}')
print(f'训练集 Recall: {recall:.4f}')
final_score = 0.6 * f1 + 0.2 * accuracy + 0.2 * recall
print(f'加权最终得分: {final_score:.4f}')

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

训练集 Accuracy: 0.7885
训练集 F1-score: 0.8181
训练集 Recall: 0.9361
加权最终得分: 0.8358


# 错误样本分析
分析模型预测错误的前10个样本，便于人工检查模型常见误判类型和后续优化方向。

In [ ]:

# 如果 'pred' 不存在，提醒用户先运行推理代码
if "pred" not in train_df.columns:
    print("请先运行模型推理代码，生成 train_df['pred'] 列！")
else:
    # 找出预测错误的样本
    wrong = train_df[train_df["pred"] != train_df["label"]]
    # 展示前10条错误样本
    print(wrong[["q1", "q2", "label", "pred"]].head(10))
    # 统计错误类型分布
    print("错误类型分布：")
    print(wrong["label"].value_counts())
    # 统计正例被误判为负例、负例被误判为正例的数量
    fp = ((wrong["label"] == 0) & (wrong["pred"] == 1)).sum()
    fn = ((wrong["label"] == 1) & (wrong["pred"] == 0)).sum()
    print(f"负例被误判为正例（假阳性）: {fp}")
    print(f"正例被误判为负例（假阴性）: {fn}")

In [ ]:
# 筛选负例被误判为正例的样本
wrong_neg = wrong[wrong['label'] == 0]

# 计算这些样本的文本相似度
wrong_neg['similarity'] = wrong_neg.apply(
    lambda row: util.cos_sim(
        model.encode(row['q1'], convert_to_tensor=True),
        model.encode(row['q2'], convert_to_tensor=True)
    ).item(), axis=1
)

# 按相似度降序排序
wrong_neg_sorted = wrong_neg.sort_values(by='similarity', ascending=False)

# 展示相似度最高的前10个样本
print("相似度最高的负例被误判为正例样本：")
print(wrong_neg_sorted[["q1", "q2", "label", "pred", "similarity"]].head(10))

In [25]:
def extract_svo_ltp4(result):
    """
    从 LTP pipeline 返回结构中提取主谓宾
    结构通常为: (words, pos, dep)
    """
    if not isinstance(result, (list, tuple)) or len(result) < 3:
        return []  # 防止 result 为空或结构错误

    words, pos, dep = result[:3]  # 取前三项
    svos = []
    for i, (head, rel, dep_idx) in enumerate(dep):
        if rel == 'SBV':
            verb_idx = head - 1
            if 0 <= verb_idx < len(words):
                subject = words[i]
                verb = words[verb_idx]
                # 查找宾语
                for j, (h2, r2, d2) in enumerate(dep):
                    if r2 == 'VOB' and h2 - 1 == verb_idx:
                        obj = words[j]
                        svos.append((subject, verb, obj))
    return svos

## 替换 HanLP 依存句法分析为 LTP，并实现主谓宾结构对比辅助二次筛选
本节将使用 LTP 进行依存句法分析，抽取主谓宾三元组，并对比两句主谓宾结构，辅助判断模型误判的负例是否可二次筛选为正例。

In [27]:
# 优化主谓宾二次筛选逻辑，使用批量处理
from tqdm import tqdm  # 用于显示进度条

# 定义主谓宾比重计算函数
def calculate_svo_weight(svo1, svo2):
    """
    计算主谓宾比重得分
    主语比重: 0.4
    谓语比重: 0.4
    宾语比重: 0.2
    """
    score = 0
    for (sub1, verb1, obj1), (sub2, verb2, obj2) in zip(svo1, svo2):
        if sub1 == sub2:
            score += 0.4
        if verb1 == verb2:
            score += 0.4
        if obj1 == obj2:
            score += 0.2
    return score

# 使用 LTP 对负例进行主谓宾抽取和筛选（批量处理）
filtered_neg = []
for idx, row in tqdm(wrong_neg.iterrows(), total=len(wrong_neg), desc="Processing Negative Samples"):
    res1 = ltp.pipeline([row['q1']])[0]
    res2 = ltp.pipeline([row['q2']])[0]
    svo1 = extract_svo_ltp4(res1)
    svo2 = extract_svo_ltp4(res2)
    weight = calculate_svo_weight(svo1, svo2)
    if weight >= 0.4:  # 比重大于等于 0.4 则保留为正例
        filtered_neg.append(idx)

# 更新预测结果
train_df.loc[filtered_neg, 'pred'] = 1

# 重新计算指标
accuracy = accuracy_score(train_df['label'], train_df['pred'])
f1 = f1_score(train_df['label'], train_df['pred'])
recall = recall_score(train_df['label'], train_df['pred'])
print(f'二次筛选后 Accuracy: {accuracy:.4f}')
print(f'二次筛选后 F1-score: {f1:.4f}')
print(f'二次筛选后 Recall: {recall:.4f}')
final_score = 0.6 * f1 + 0.2 * accuracy + 0.2 * recall
print(f'二次筛选后加权最终得分: {final_score:.4f}')

Processing Negative Samples: 100%|██████████| 1519/1519 [02:15<00:00, 11.21it/s]

二次筛选后 Accuracy: 0.7885
二次筛选后 F1-score: 0.8181
二次筛选后 Recall: 0.9361
二次筛选后加权最终得分: 0.8358
